In [79]:
import pandas as pd
import numpy as np
from numpy import nan


In [80]:
df = pd.read_csv("jobs14:03:24.csv")

In [81]:
df.head()

,Unnamed: 0,title,company,salary,location,company_rating,description
0,0,Junior Data Scientist (NYC or Remote),ViacomCBS,NaN,"Remote in New York, NY+1 location",3.9,ViacomCBS (NASDAQ:\nVIAC; VIACA) is a leading ...
1,1,"Associate Data Scientist, Content Analytics",Spotify,NaN,Remote in Oregon+12 locations,4.3,"Data, Research & Insights\nData Science\nSpoti..."
2,2,"Data Scientist, TikTok Creation and Consumption",TikTok,NaN,"San Francisco, CA+34 locations",3.5,TikTok is the leading destination for short-fo...
3,3,"Data Scientist, Game Analytics & Strategy",the NBA,NaN,"New York, NY 10022 (Midtown area)+1 location",4.2,"At the NBA, we’re passionate about growing and..."
4,4,Data Scientist and Analytics Specialist,Delta,NaN,"Atlanta, GA+4 locations",4.2,"United States, Georgia, Atlanta\nProduct Manag..."


In [82]:
df.shape

(450, 7)

We will start to work from salary column. Goal here is to have 
1) mean value for the salary if they have a salary estimation
2) or leave it as NaN

First Let's figure out
- How many NaN value we have
- How many value that we can use
- structure of non NaN values
- extract only numbers

In [83]:
print('we have', df['salary'].isna().sum(), 'Nan value for the salary column')
print('that indicates that we have only 56 values to use')

we have 394 Nan value for the salary column
that indicates that we have only 56 values to use


In [84]:
salary = df['salary']
salary_notnull = salary[salary.notnull()]
print(salary_notnull)


11           $110,000 - $140,000 a year
12            $72,272 - $176,300 a year
17            $92,700 - $160,000 a year
21                      $110,500 a year
23            $83,690 - $160,000 a year
26            $70,400 - $117,400 a year
29            $76,000 - $148,000 a year
33            $83,690 - $160,000 a year
35           $120,000 - $223,000 a year
36            $72,272 - $176,300 a year
39     Estimated $47.7K – $60.3K a year
41            $90,000 - $150,000 a year
46            $90,000 - $150,000 a year
55            $86,389 - $116,788 a year
57            $70,400 - $117,400 a year
58                       $95,000 a year
69            $65,072 - $117,129 a year
71           $125,000 - $175,000 a year
72            $90,000 - $150,000 a year
75           $118,248 - $127,208 a year
93                       $98,000 a year
96             $4,372 - $10,792 a month
104           $90,000 - $161,000 a year
112           $90,000 - $161,000 a year
117           $96,400 - $144,600 a year


Now let's extract numbers only from the salary columns

In [85]:
def str_contains_n(str):
    num_str = "0123456789"
    word_list = []
    words = str.split(" ")
    for word in words:
        word_has_n = False
        for char in word:
            if char in num_str:
                word_has_n = True
        if word_has_n:
            word_list.append(word)
    result = '-'.join(word_list)
    return result

In [86]:
salary = salary.apply(lambda x: str_contains_n(x) if isinstance(x, str) else None)
salary = salary.apply(lambda x: x.lower().replace("$","") if x != None else None)
salary = salary.apply(lambda x: x.lower().replace("a year","") if x != None else None) 
salary = salary.apply(lambda x: x.lower().replace("Estimated","") if x != None else None)
k_unit = salary.apply(lambda x: True if "k" in x!= None and x.lower() else False)
min = salary.apply(lambda x: x.lower().replace("$","").replace("k","") if x != None else None)
max =  

# minus_kd = salary.apply(lambda x: x.remove('K').remove('$'))
# min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))